Gabor ML - Problem 16

-Clarice Mottet

Outline:
-generate training data
    -x_i multivariate normal distribution (mean zero, covariance diagonal (1/1...d))
    -y_i sum(x_i) + e_i where e_i standard normal random variable
-measure performance by mean squared error R(w) = E(wTX-Y)^2
-write program that learns vector w by OLS
-write program that learns vector w by GD
-write program that learns vector w by SGD
-compare performance by mean squared error and running time for multiple n and d and step size n_t

In [3]:
#libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

path_out_ = r'/home/clarice/Documents/VSCode/Term2_Gabor_ML/homework4/GaborML_problem16/outputs'
random.seed(123)

In [26]:
#functions

def generate_data(n, d):

    #covariance matrix
    cov = np.identity(d)    
    for i in range(d):
        cov[i,i] = 1/(i+1)

    #x
    x = np.random.multivariate_normal(np.zeros(d), cov, n)

    #y
    y = np.zeros(n)
    e = np.random.normal(0, 1, n)
    for i in range(n):
        y[i] = sum(x[i]) + e[i]

    return x, y

def solve_OLS(x, y):
    n = len(x)
    d = len(x[0])

    matrix_xx = np.zeros((d,d))
    for i in range(n):
        add_matrix = np.outer(x[i], x[i].T)
        matrix_xx = np.add(matrix_xx, add_matrix)
    matrix_xx = np.multiply(1/n, matrix_xx)
    matrix_xx = np.linalg.inv(matrix_xx)

    vec_xy = np.zeros(d)
    for i in range(n):
        add_vec = np.multiply(x[i], y[i])
        vec_xy = np.add(vec_xy, add_vec)
    vec_xy = np.multiply(1/n, vec_xy)

    w = np.dot(matrix_xx, vec_xy)
    return w


In [51]:
n = 2
d = 2

x = np.identity(n)
x[0,0] = 0.5
y = np.zeros(n)
y[0] = .2
y[1] = .8

# x, y = generate_data(n, d)
print(x)
# print(y)

print("------------")
# x_t = np.transpose(x)
# print(x_t)

x_T = x.T
print(x_T)

print("------------")

matrix_xx = np.zeros((d,d))
for i in range(n):
    add_matrix = np.outer(x[i], x[i].T)
    matrix_xx = np.add(matrix_xx, add_matrix)
matrix_xx = np.multiply(1/n, matrix_xx)

matrix_xx = np.linalg.inv(matrix_xx)
print(matrix_xx)

print("------------")

vec_xy = np.zeros(d)
for i in range(n):
    add_vec = np.multiply(x[i], y[i])
    vec_xy = np.add(vec_xy, add_vec)
vec_xy = np.multiply(1/n, vec_xy)
print(vec_xy)

print("------------")

w = np.dot(matrix_xx, vec_xy)
print(w)



[[0.5 0. ]
 [0.  1. ]]
------------
[[0.5 0. ]
 [0.  1. ]]
------------
[[8. 0.]
 [0. 2.]]
------------
[0.05 0.4 ]
------------
[0.4 0.8]
